# 🛡️ Robust Fine-Tuning: Qwen2.5-Coder for Linux Kernel

This is an improved, robust version of the fine-tuning script designed to:
1. **Auto-save** progress to Google Drive to prevent data loss
2. **Resume** capability if the runtime disconnects
3. **Safe Export** of GGUF models

## ✅ Prerequisites
1. Upload your `premium_reasoning.jsonl` to Google Drive root folder.
2. Set Runtime to **T4 GPU** (Runtime → Change runtime type).


In [ ]:
# 1️⃣ Mount Google Drive & Setup Paths
from google.colab import drive
import os

print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

# --- CONFIGURATION ---
DRIVE_ROOT = "/content/drive/MyDrive"
DATA_FILENAME = "premium_reasoning.jsonl"
DATA_PATH = os.path.join(DRIVE_ROOT, DATA_FILENAME)

# Where we will save the trained adapters PERMANENTLY
OUTPUT_DIR_DRIVE = os.path.join(DRIVE_ROOT, "kernel_reviewer_adapters")

# Where we will save the GGUF models PERMANENTLY
GGUF_DIR_DRIVE = os.path.join(DRIVE_ROOT, "kernel_reviewer_gguf")

print(f"\n✅ Configuration:")
print(f"   Data: {DATA_PATH}")
print(f"   Adapter Backup: {OUTPUT_DIR_DRIVE}")
print(f"   GGUF Backup: {GGUF_DIR_DRIVE}")

if not os.path.exists(DATA_PATH):
    print(f"\n❌ WARNING: Data file not found at {DATA_PATH}")
    print("   Please upload it to your Drive root before proceeding!")
else:
    print("\n✅ Data file found!")

In [ ]:
# 2️⃣ Install Unsloth & Dependencies
print("⏳ Installing libraries (this takes ~2 mins)...")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install transformers datasets trl accelerate peft bitsandbytes -q

import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    !pip install flash-attn -q

print("✅ Libraries installed!")

In [ ]:
# 3️⃣ Load Base Model (Qwen2.5-Coder-7B)
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # Auto detection
load_in_4bit = True

print(f"🤖 Loading Qwen2.5-Coder-7B (4-bit)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print("✅ Model prepared!")

In [ ]:
# 4️⃣ Prepare Data
from datasets import load_dataset

alpaca_prompt = """### Instruction
{}

### Input
{}

### Response
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

print(f"📊 Loading data from {DATA_PATH}...")
dataset = load_dataset("json", data_files=DATA_PATH, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print(f"✅ Data loaded! {len(dataset)} examples.")

In [ ]:
# 5️⃣ Train with Auto-Save to Drive
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import shutil

# Working directory (local to Colab VM for speed)
LOCAL_OUTPUT_DIR = "/content/kernel_reviewer"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for full run
        max_steps = 60, # Testing! Remove this and use num_train_epochs = 1 for full run
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = LOCAL_OUTPUT_DIR,
    ),
)

print("🚀 Starting Training...")
trainer_stats = trainer.train()

# --- CRITICAL AUTO-SAVE STEP ---
print("\n💾 Training complete! Saving adapters to Google Drive...")

# 1. Save locally first
model.save_pretrained(LOCAL_OUTPUT_DIR)
tokenizer.save_pretrained(LOCAL_OUTPUT_DIR)

# 2. Copy to Drive immediately
if os.path.exists(OUTPUT_DIR_DRIVE):
    shutil.rmtree(OUTPUT_DIR_DRIVE) # Clear old version
shutil.copytree(LOCAL_OUTPUT_DIR, OUTPUT_DIR_DRIVE)

print(f"✅ SAFE! Adapters backed up to: {OUTPUT_DIR_DRIVE}")

# Also print stats
print(f"Global Step: {trainer_stats.global_step}")
print(f"Training Loss: {trainer_stats.training_loss}")

In [ ]:
# 6️⃣ Export to GGUF (Safe Mode)
# We separate this step so if it crashes, your training data is already safe in Drive in Step 5.

print("📥 Starting GGUF Export (q4_k_m)...")
print("Note: This might crash low-RAM runtimes, but your adapters are safe in Drive.")

# Define GGUF output in Drive directly if possible, or copy after
LOCAL_GGUF_DIR = "/content/gguf_export"
if not os.path.exists(LOCAL_GGUF_DIR):
    os.makedirs(LOCAL_GGUF_DIR)

# Export
model.save_pretrained_gguf(
    LOCAL_GGUF_DIR,
    tokenizer,
    quantization_method = "q4_k_m"
)

print("📦 Export done locally! Moving to Drive...")

# Move GGUF files to Drive
if not os.path.exists(GGUF_DIR_DRIVE):
    os.makedirs(GGUF_DIR_DRIVE)

import glob
for file in glob.glob(f"{LOCAL_GGUF_DIR}/*.gguf"):
    shutil.copy(file, GGUF_DIR_DRIVE)
    print(f"   Saved: {file} -> {GGUF_DIR_DRIVE}")

print("\n✅ ALL DONE! Checks your Drive folder 'kernel_reviewer_gguf' for the model file.")